# Text 생성 및 Prompt 예제

HyperCLOVAX‑SEED‑Text‑Instruct‑0.5B는 지시문 기반 텍스트-투-텍스트 모델로, 한국어 언어 및 문화 이해에 뛰어난 성능을 보입니다. 유사한 규모의 외부 경쟁 모델과 비교했을 때 수학적 성능이 향상되고, 한국어 능력이 크게 개선되었습니다. 이 모델은 HyperCLOVAX 시리즈에서 현재 출시된 모델 중 가장 작은 모델이며, 에지 디바이스와 같은 리소스가 제한된 환경에 적합한 경량 솔루션입니다. 최대 4K 토큰의 컨텍스트 길이를 지원하며, 다양한 작업에 적용 가능한 다목적 소형 모델입니다.

- Hugging Face 로그인 및 토큰 생성

https://huggingface.co/settings/tokens 에서 Read 권한의 토큰을 생성하세요.

- 모델 접근 권한 요청

모델 페이지: https://huggingface.co/naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B

"Access repository" 버튼을 클릭해서 권한 요청

In [2]:
from dotenv import load_dotenv
load_dotenv() # read local .env file

True

In [8]:
import os
token = os.getenv("HUGGING_FACE_TOKEN")

In [9]:
from huggingface_hub import login

# 토큰 입력 (직접 입력하거나 환경변수/Colab secret 사용 가능)
login(token=token)

In [10]:
from transformers import AutoModelForCausalLM, AutoProcessor, AutoTokenizer

model_name = "naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/764 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.17G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/11.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

## 문법 수정

SYSTEM : 당신은 문장을 받게 될 것이며, 당신의 임무는 그것을 표준 한국어로 변환하는 것입니다.  
USER :   
안갔어 시장에 그녀는.

In [11]:
def generate_response(system_content, user_content, max_length=1024, repetition_penalty=1.2):
    """
    HyperCLOVAX 모델을 사용하여 응답을 생성하는 함수

    Args:
        system_content (str): 시스템 메시지 내용
        user_content (str): 사용자 메시지 내용
        max_length (int): 최대 생성 길이 (기본값: 1024)
        repetition_penalty (float): 반복 패널티 (기본값: 1.2)

    Returns:
        str: 생성된 응답 텍스트
    """
    chat = [
        {"role": "tool_list", "content": ""},
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content},
    ]

    inputs = tokenizer.apply_chat_template(chat, add_generation_prompt=True, return_dict=True, return_tensors="pt")
    inputs = inputs.to("cuda")

    output_ids = model.generate(
        **inputs,
        max_length=max_length,
        repetition_penalty=repetition_penalty,
        eos_token_id=tokenizer.eos_token_id,
    )

    output_text = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0]

    # 필요시 <|endofturn|>, <|stop|> 등에서 자르기
    for stop_str in ["<|endofturn|>", "<|stop|>"]:
        if stop_str in output_text:
            output_text = output_text.split(stop_str)[0]

    return output_text

In [12]:
# 문법 수정 예시
system_msg = "당신은 문장을 받게 될 것이며, 당신의 임무는 그것을 표준 한국어로 변환하는 것입니다."
user_msg = "안갔어 시장에 그녀는."

result = generate_response(system_msg, user_msg)
print(result)

tool_list

system
당신은 문장을 받게 될 것이며, 당신의 임무는 그것을 표준 한국어로 변환하는 것입니다.
user
안갔어 시장에 그녀는.
assistant
시장에 안 갔어요? 그녀는요?


## 구조화되지 않은 데이터의 구문 분석
SYSTEM : 구조화되지 않은 데이터가 제공되며 이를 CSV 형식으로 구문 분석하는 작업이 수행됩니다.  
USER :   
최근 발견된 행성 구크럭스(Goocrux)에서는 많은 과일이 발견됐다. 그곳에서 자라는 네오스키즐이 있는데, 보라색이고 사탕 맛이 납니다. 회색 빛이 도는 파란색 과일이고 매우 시큼하며 레몬과 약간 비슷한 로헤클(loheckles)도 있습니다. 포유닛은 밝은 녹색을 띠며 단맛보다 풍미가 더 좋습니다. 네온 핑크색 맛과 솜사탕 같은 맛이 나는 루프노바도 많이 있습니다. 마지막으로 글로울(glowls)이라는 과일이 있는데, 이 과일은 신맛과 부식성이 있는 매우 신맛과 쓴맛이 나며 옅은 오렌지색을 띠고 있습니다.

In [13]:
system_msg = "구조화되지 않은 데이터가 제공되며 이를 CSV 형식으로 구문 분석하는 작업이 수행됩니다."

user_msg = """
최근 발견된 행성 구크럭스(Goocrux)에서는 많은 과일이 발견됐다. 그곳에서 자라는 네오스키즐이 있는데, 보라색이고 사탕 맛이 납니다.
회색 빛이 도는 파란색 과일이고 매우 시큼하며 레몬과 약간 비슷한 로헤클(loheckles)도 있습니다. 포유닛은 밝은 녹색을 띠며 단맛보다 풍미가 더 좋습니다.
네온 핑크색 맛과 솜사탕 같은 맛이 나는 루프노바도 많이 있습니다. 마지막으로 글로울(glowls)이라는 과일이 있는데,
이 과일은 신맛과 부식성이 있는 매우 신맛과 쓴맛이 나며 옅은 오렌지색을 띠고 있습니다.
이 데이터를 과일명, 색상, 맛으로 분석해 주세요.
"""

result = generate_response(system_msg, user_msg)
print(result)

tool_list

system
구조화되지 않은 데이터가 제공되며 이를 CSV 형식으로 구문 분석하는 작업이 수행됩니다.
user

최근 발견된 행성 구크럭스(Goocrux)에서는 많은 과일이 발견됐다. 그곳에서 자라는 네오스키즐이 있는데, 보라색이고 사탕 맛이 납니다.
회색 빛이 도는 파란색 과일이고 매우 시큼하며 레몬과 약간 비슷한 로헤클(loheckles)도 있습니다. 포유닛은 밝은 녹색을 띠며 단맛보다 풍미가 더 좋습니다.
네온 핑크색 맛과 솜사탕 같은 맛이 나는 루프노바도 많이 있습니다. 마지막으로 글로울(glowls)이라는 과일이 있는데,
이 과일은 신맛과 부식성이 있는 매우 신맛과 쓴맛이 나며 옅은 오렌지색을 띠고 있습니다.
이 데이터를 과일명, 색상, 맛으로 분석해 주세요.

assistant
```json
{
  "fruits": [
    {
      "name": "보라색",
      "color": "파란색",
      "taste": "시큼한"
    },
    {
      "name": "로헤클",
      "color": "회색빛이 도는 파란색",
      "taste": "레몬과 비슷하게 신맛"
    },
    {
      "name": "포유닛",
      "color": "밝은 녹색",
      "taste": "단맛보다는 풍미"
    },
    {
      "name": "루프노바",
      "color": "네온 핑크색",
      "taste": "솜사탕 같이 달콤하고 시큼함"
    },
    {
      "name": "글로울",
      "color": "신맛과 부식성을 가진 옅은 오렌지색"
      "taste": "신맛과 쓴맛"
    }
  ]
}
```

위 JSON 구조에 따라 각 과일의 이름, 색상 및 맛을 정리했습니다. 이 정보를 통해 다양한 과일에 대한 세부 사항을 쉽게 파악할 수 있도록 구성하였습니다.

각 과일을 구분하여 명확히 나타내기 위해 다음과 같은 방식으

## Keyword 추출

In [14]:
text = """
"블랙 온 블랙 도자기(Black-on-Black ware)는 뉴멕시코 북부의 푸에블로 원주민 도자기 예술가들이 개발한 20세기 및 21세기 도자기 전통입니다.
전통적인 환원 소성 블랙웨어는 푸에블로 예술가들에 의해 수세기 동안 만들어졌습니다.
지난 세기의 흑색 자기는 표면이 매끄럽고 선택적 버니싱이나 내화 슬립을 적용하여 디자인을 적용한 제품입니다.
또 다른 스타일은 디자인을 조각하거나 절개하고 융기된 부분을 선택적으로 연마하는 것입니다.
여러 세대에 걸쳐 Kha'po Owingeh와 P'ohwhóge Owingeh 푸에블로의 여러 가족은 여주인 도예가들로부터 전수받은 기술을 사용하여 검은 바탕에 검은 도자기를 만들어 왔습니다.
다른 푸에블로 출신의 예술가들도 검정색 바탕에 검정색 도자기를 제작했습니다. 몇몇 현대 예술가들은 조상의 도자기를 기리는 작품을 만들었습니다."
"""

instructions="텍스트 블록이 제공되며, 당신의 임무는 텍스트 블록에서 키워드 목록을 추출하는 것입니다."

result = generate_response(instructions, text)
print(result)

tool_list

system
텍스트 블록이 제공되며, 당신의 임무는 텍스트 블록에서 키워드 목록을 추출하는 것입니다.
user

"블랙 온 블랙 도자기(Black-on-Black ware)는 뉴멕시코 북부의 푸에블로 원주민 도자기 예술가들이 개발한 20세기 및 21세기 도자기 전통입니다.
전통적인 환원 소성 블랙웨어는 푸에블로 예술가들에 의해 수세기 동안 만들어졌습니다.
지난 세기의 흑색 자기는 표면이 매끄럽고 선택적 버니싱이나 내화 슬립을 적용하여 디자인을 적용한 제품입니다.
또 다른 스타일은 디자인을 조각하거나 절개하고 융기된 부분을 선택적으로 연마하는 것입니다.
여러 세대에 걸쳐 Kha'po Owingeh와 P'ohwhóge Owingeh 푸에블로의 여러 가족은 여주인 도예가들로부터 전수받은 기술을 사용하여 검은 바탕에 검은 도자기를 만들어 왔습니다.
다른 푸에블로 출신의 예술가들도 검정색 바탕에 검정색 도자기를 제작했습니다. 몇몇 현대 예술가들은 조상의 도자기를 기리는 작품을 만들었습니다."

assistant
- 푸에블로 도자기 예술: 
   - 전통적인 방식  
   - 디자인 적용 방법 (버니싱 또는 내화슬리퍼 사용)   
   - 다양한 기술  

- 현대 예술가들:
   - 조상들의 작품 기리기 

더 많은 정보를 알고 싶으시면 말씀해 주세요!


## 감성 분류기
- 한개의 text 감성 분석

In [15]:
 instruction="당신은 텍스트를 입력 받게 될 것이고, 당신의 임무는 텍스트의 감정을 긍정적, 중립적, 부정적으로 분류하는 것입니다."
 input="나는 새로운 배트맨 영화가 좋습니다!"

result = generate_response(instruction,  input)
print(result)

tool_list

system
당신은 텍스트를 입력 받게 될 것이고, 당신의 임무는 텍스트의 감정을 긍정적, 중립적, 부정적으로 분류하는 것입니다.
user
나는 새로운 배트맨 영화가 좋습니다!
assistant
긍정적인 감정입니다. "새로운"라는 표현이 긍정적인 느낌을 주고 있습니다. 또한 "좋습니다!"라고 말함으로써 열정과 만족감을 나타내고 있습니다.


## 회의록 요약

SYSTEM : 회의록이 제공되며 귀하의 임무는 다음과 같이 회의를 요약하는 것입니다.  

 -토론의 전반적인 요약  
 -행동항목(무엇을 해야 하는지, 누가 하는지)  
 -해당하는 경우 다음 회의에서 더 자세히 논의해야 할 주제 목록입니다.  

In [16]:
meeting_minutes = """
회의 날짜: 2050년 3월 5일
 미팅 시간: 오후 2시
 위치: 은하계 본부 회의실 3B

 참석자:
 - 캡틴 스타더스트
 - 퀘이사 박사
 - 레이디 네뷸라
 - 초신성 경
 - 혜성 씨

 오후 2시 5분에 캡틴 스타더스트가 회의를 소집했습니다.

 1. 새로운 팀원인 Ms. Comet에 대한 소개와 환영 인사

 2. Planet Zog에 대한 우리의 최근 임무에 대한 토론
 - 캡틴 스타더스트: "전반적으로 성공했지만, Zogians와의 의사소통이 어려웠습니다. 언어 능력을 향상시켜야 합니다."
 - 퀘이사 박사: "동의합니다. 즉시 Zogian-영어 사전 작업을 시작하겠습니다."
 - Lady Nebula: "Zogian 음식은 말 그대로 이 세상의 것이 아니었습니다! 우리는 배에서 Zogian 음식의 밤을 갖는 것을 고려해야 합니다."

 3. 7구역 우주 해적 문제 해결
 - 초신성 경: "이 해적들을 처리하려면 더 나은 전략이 필요합니다. 그들은 이번 달에 이미 세 척의 화물선을 약탈했습니다."
 - 스타더스트 선장: "그 지역의 순찰을 늘리는 것에 대해 스타빔 제독과 이야기하겠습니다.
 - 퀘이사 박사: "저는 우리 함선이 해적의 발각을 피하는 데 도움이 될 수 있는 새로운 은폐 기술을 연구하고 있습니다. 프로토타입을 완성하려면 몇 주가 더 필요할 것입니다."

 4. 연례 은하계 베이크오프 검토
 - Lady Nebula: "우리 팀이 대회에서 2위를 했다는 소식을 전해드리게 되어 기쁩니다! 우리 화성 머드 파이가 대박을 쳤어요!"
 - 혜성 씨: "내년에는 1위를 목표로 합시다. 제 생각에는 승자가 될 수 있을 것 같은 주피터 젤로의 비법이 있습니다."

 5. 다가오는 자선 모금 행사 계획
 - Captain Stardust: "Intergalactic Charity Bazaar 부스에 대한 창의적인 아이디어가 필요합니다."
 - Sir Supernova: "'Dunk the Alien' 게임은 어때요? 외계인 복장을 한 자원봉사자에게 사람들이 물 풍선을 던지게 할 수 있어요."
 - 퀘이사 박사: "승자에게 상금을 주는 '별 이름을 지어라' 퀴즈 게임을 준비할 수 있어요."
 - Lady Nebula: "좋은 아이디어입니다, 여러분. 이제 보급품을 모으고 게임을 준비합시다."

 6. 다가오는 팀 빌딩 수련회
 - Comet 씨: "Moon Resort and Spa에서 팀워크를 다지는 휴양지를 제안하고 싶습니다. 최근 임무를 마친 후 유대감을 형성하고 휴식을 취할 수 있는 좋은 기회입니다."
 - 캡틴 스타더스트: "환상적인 생각이군요. 예산을 확인해 보고 실현할 수 있는지 알아보겠습니다."

 7. 차기회의 안건
 - Zogian-English 사전 업데이트 (Dr. Quasar)
 - 클로킹 기술 진행 보고서(퀘이사 박사)
 - 7번 구역 순찰 강화 결과(캡틴 스타더스트)
 - 은하계 자선 바자회 최종 준비(전체)

 회의가 오후 3시 15분에 연기되었습니다. 다음 회의는 2050년 3월 19일 오후 2시에 은하계 본부 회의실 3B에서 열릴 예정입니다.
"""
instruction = """
          회의록이 제공되며 귀하의 임무는 다음과 같이 회의를 요약하는 것입니다.
             -토론의 전반적인 요약
             -행동항목(무엇을 해야 하는지, 누가 하는지)
             -해당하는 경우 다음 회의에서 더 자세히 논의해야 할 주제 목록입니다.
      """
result = generate_response(instruction, meeting_minutes)
print(result)

tool_list

system

          회의록이 제공되며 귀하의 임무는 다음과 같이 회의를 요약하는 것입니다.
             -토론의 전반적인 요약
             -행동항목(무엇을 해야 하는지, 누가 하는지)
             -해당하는 경우 다음 회의에서 더 자세히 논의해야 할 주제 목록입니다.
      
user

회의 날짜: 2050년 3월 5일
 미팅 시간: 오후 2시
 위치: 은하계 본부 회의실 3B

 참석자:
 - 캡틴 스타더스트
 - 퀘이사 박사
 - 레이디 네뷸라
 - 초신성 경
 - 혜성 씨

 오후 2시 5분에 캡틴 스타더스트가 회의를 소집했습니다.

 1. 새로운 팀원인 Ms. Comet에 대한 소개와 환영 인사

 2. Planet Zog에 대한 우리의 최근 임무에 대한 토론
 - 캡틴 스타더스트: "전반적으로 성공했지만, Zogians와의 의사소통이 어려웠습니다. 언어 능력을 향상시켜야 합니다."
 - 퀘이사 박사: "동의합니다. 즉시 Zogian-영어 사전 작업을 시작하겠습니다."
 - Lady Nebula: "Zogian 음식은 말 그대로 이 세상의 것이 아니었습니다! 우리는 배에서 Zogian 음식의 밤을 갖는 것을 고려해야 합니다."

 3. 7구역 우주 해적 문제 해결
 - 초신성 경: "이 해적들을 처리하려면 더 나은 전략이 필요합니다. 그들은 이번 달에 이미 세 척의 화물선을 약탈했습니다."
 - 스타더스트 선장: "그 지역의 순찰을 늘리는 것에 대해 스타빔 제독과 이야기하겠습니다.
 - 퀘이사 박사: "저는 우리 함선이 해적의 발각을 피하는 데 도움이 될 수 있는 새로운 은폐 기술을 연구하고 있습니다. 프로토타입을 완성하려면 몇 주가 더 필요할 것입니다."

 4. 연례 은하계 베이크오프 검토
 - Lady Nebula: "우리 팀이 대회에서 2위를 했다는 소식을 전해드리게 되어 기쁩니다! 우리 화성 머드 파이가 대박을 쳤어요!"
 - 혜성 씨: "내년에는 1위를 목표로 합시다. 제 

## 이모티콘 번역
SYSTEM : 텍스트가 제공되며, 귀하의 임무는 이를 이모티콘으로 번역하는 것입니다. 일반 텍스트를 사용하지 마십시오. 이모티콘만으로 최선을 다하세요.  
USER : 인공지능은 큰 가능성을 지닌 기술이다.

In [17]:
instruction = "텍스트가 제공되며, 귀하의 임무는 이를 이모티콘으로 번역하는 것입니다. 일반 텍스트를 사용하지 마십시오. 이모티콘만으로 최선을 다하세요."
content = "인공지능은 큰 가능성을 지닌 기술이다."

result = generate_response(instruction, content)
print(result)

tool_list

system
텍스트가 제공되며, 귀하의 임무는 이를 이모티콘으로 번역하는 것입니다. 일반 텍스트를 사용하지 마십시오. 이모티콘만으로 최선을 다하세요.
user
인공지능은 큰 가능성을 지닌 기술이다.
assistant
🤖✨💡🌐🚀


## 번역

In [18]:
instruction = "당신은 영어로 된 문장을 받게 될 것이고, 당신의 임무는 그것을 한국어와 동시에 일본어로 번역하는 것입니다."
content = "My name is Jane. What is yours?"

result = generate_response(instruction, content)
print(result)

tool_list

system
당신은 영어로 된 문장을 받게 될 것이고, 당신의 임무는 그것을 한국어와 동시에 일본어로 번역하는 것입니다.
user
My name is Jane. What is yours?
assistant
안녕하세요! 저는 제인입니다.

제 이름은 뭐예요?

- Korean: 내 이름은 제인이에요.
- Japanese: 私の名前はジェインです。お名前は何ですか？


## 실습: 위의 Prompt 내용을 수정해 가며 api 실행